In [1]:
import pandas as pd
import numpy as np
import stumpy as sp
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as dates
%matplotlib widget  
from ipywidgets import *
from matplotlib.patches import Rectangle
from matplotlib.pyplot import cm

In [13]:
battery_data = pd.concat([pd.read_csv('data/battery_data/2020-{}.csv'.format(month)) for month in range(4, 10)])
capacity_data = pd.read_csv('data/testbench_results/testbench_results_first3months.csv')
battery_data['datetime'] = pd.to_datetime(battery_data['timestamp'], unit='s')
key='voltage'

In [14]:
subset = battery_data[battery_data['timestamp']<datetime.datetime(2020, 10, 1, 0, 0, 0).timestamp()]
subset = subset[subset['timestamp']>datetime.datetime(2020, 4, 1, 0, 0, 0).timestamp()]
subset = subset.set_index('datetime')
subset_ma = subset.resample("1H").median()
print(subset_ma[key].shape, subset[key].shape)
plt.figure()
plt.plot(subset_ma.index, subset_ma[key])
plt.show()

(4381,) (14347672,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
m=17
mp = sp.stump(subset_ma[key], m)

In [16]:
plt.figure()
plt.plot(mp[:, 0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
all_chain_set, unanchored_chain = sp.allc(mp[:, 2], mp[:, 3])

In [18]:
plt.figure()
start_times = []
plt.plot(subset_ma[key], linewidth=1, color='black')
for i in range(unanchored_chain.shape[0]):
    y = subset_ma[key].iloc[unanchored_chain[i]:unanchored_chain[i]+m]
    x = y.index.values
    start_times.append(x[0])
    plt.plot(x, y, linewidth=3)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.figure()
means = []
stdevs = []
color=cm.cividis(np.linspace(0,1,unanchored_chain.shape[0]))
for i in range(unanchored_chain.shape[0]):
    y = subset_ma[key].iloc[unanchored_chain[i]:unanchored_chain[i]+m]
    means.append(np.mean(y))
    stdevs.append(np.std(y))
    # x = y.index.values
    plt.plot(x, y, linewidth=3, color=color[i], alpha=0.3)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# plt.figure()
capacity_data['datetime'] = pd.to_datetime(capacity_data['timestamp'], unit='s')
color1 = plt.cm.viridis(0)
color2 = plt.cm.viridis(0.8)
fig, host = plt.subplots(figsize=(8,5))

par1 = host.twinx()
host.set_ylim(-11.27, -11.42)
host.set_xlabel("Time")
host.set_ylabel("Capacity")
par1.set_ylabel("Mean motif value")
p1, = host.plot(capacity_data['datetime'], capacity_data['Q_discharge'], color=color1, label="Capacity")
p2 = par1.errorbar(start_times, means, yerr=stdevs, color=color2, label="Mean motif value")
lns = [p1, p2]
host.legend(handles=lns, loc='best')
# plt.plot(start_times, means, capacity_data['datetime'], capacity_data['Q_discharge'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
class timeseries_chain_motif:
    def __init__(self, subset_ma, key, m):
        self.subset_ma = subset_ma
        self.key = key
        self.m = m
        self.mp = sp.stump(subset_ma[key], m)  
        self.all_chain_set = sp.allc(mp[:, 2], mp[:, 3])[0]
        self.unanchored_chain = sp.allc(mp[:, 2], mp[:, 3])[1]
    
    def mp_plot(self, show=True):
        if show:
            plt.figure()
        start_times = []
        if show:
            plt.plot(self.subset_ma[self.key], linewidth=1, color='black')
        for i in range(self.unanchored_chain.shape[0]):
            y = self.subset_ma[self.key].iloc[self.unanchored_chain[i]:self.unanchored_chain[i]+self.m]
            x = y.index.values
            start_times.append(x[0])
            if show:
                plt.plot(x, y, linewidth=3)
        self.start_times = start_times

    def motif_compare_plot(self, show=True):
        if show:
            plt.figure()
        means = []
        stdevs = []

        color=cm.cividis(np.linspace(0,1,self.unanchored_chain.shape[0]))
        for i in range(self.unanchored_chain.shape[0]):
            y = self.subset_ma[self.key].iloc[self.unanchored_chain[i]:self.unanchored_chain[i]+self.m]
            means.append(np.mean(y))
            stdevs.append(np.std(y))
            # x = y.index.values
            if show:
                plt.plot(x, y, linewidth=3, color=color[i], alpha=0.3)
        self.means = means
        self.stdevs = stdevs
    
    def capacity_compare_plot(self, capacity_data):
        self.mp_plot(show=False)
        self.motif_compare_plot(show=False)
        capacity_data['datetime'] = pd.to_datetime(capacity_data['timestamp'], unit='s')
        color1 = plt.cm.viridis(0)
        color2 = plt.cm.viridis(0.8)
        fig, host = plt.subplots(figsize=(8,5))
        par1 = host.twinx()
        host.set_ylim(-11.27, -11.42)
        host.set_xlabel("Time")
        host.set_ylabel("Capacity")
        par1.set_ylabel("Mean motif value")
        p1, = host.plot(capacity_data['datetime'], capacity_data['Q_discharge'], color=color1, label="Capacity")
        p2 = par1.errorbar(self.start_times, self.means, yerr=self.stdevs, color=color2, label="Mean motif value")
        lns = [p1, p2]
        host.legend(handles=lns, loc='best')

In [22]:
def subset_data(data, resample):
    subset = data.set_index('datetime')
    subset_ma = subset.resample(resample).mean()
    return subset_ma

In [12]:
tscm = timeseries_chain_motif(subset_data(battery_data, '1H'), 'voltage', 34)
tscm.capacity_compare_plot(capacity_data)

KeyError: "None of ['datetime'] are in the columns"

In [ ]:
m_vals = np.arange(4,34,4)
for m in m_vals:
    tscm = timeseries_chain_motif(subset_ma, 'voltage', m)
    tscm.capacity_compare_plot(capacity_data)
    plt.title(m)